# Построение рекомендаций соавторов
##Постановка задачи
Есть данные о соавторах и количестве их совместных публикаций (трэйн/тест выборки - результат спринта 3). По этим данным необходимо построить рекомендации для соавторов. Исходя из того, что авторы публикуют статьи как правила по одной отрасли, принято решение давать рекомендации внутри каждого класса по отдельности. 

##Алгоритм:
  1.  Для обучающего набора разбиваем авторов по классам (если у автора есть статьи из нескольких классов, то определяем его к доменирующему классу)
  2.  Внутри каждого классса строим матрицу взаимодействия авторов.
  3.  Для матрицы взаимодействий строим матрицу схожести авторов на основе косинусной близости.
  4.  Для каждого автора на основе матрицы схожести и порогового значения сходства (SIM_VAL) рекомендуем наиболее схожих соавторов.
  5.  Валидируем рекомендации на отложеной выборке (результат спринта 3) и подбираем пороговое значение параметра SIM_VAL.
  6. Для выбранного SIM_VAL строим рекомендации по всей выборки данных.
  

## Загрузка данных
Загружаем данные о совместных публикациях

In [1]:
import numpy as np
import pandas as pd
import os

path = '/content/drive/MyDrive/MADE/data/splited data'
data_train = pd.read_csv(path + '/train.csv', header=0)
data_test = pd.read_csv(path + '/test.csv', header = 0)
data_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2575786 entries, 0 to 2575785
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   author_id_1  object
 1   author_id_2  object
 2   value        int64 
dtypes: int64(1), object(2)
memory usage: 59.0+ MB


Еще раз проверяем на дубликаты

In [2]:
data_train.drop_duplicates(subset=['author_id_1', 'author_id_2'], inplace=True, ignore_index=True)

Загружаем данные о классе публикации (подготовлены в спринте 3)

In [3]:
df = pd.read_csv('/content/drive/MyDrive/MADE/data/splited data/authors_year_label_not_grouped.zip', compression='zip', header=0)
df.head()

,author_id_1,author_id_2,year,label,article_id
0,53f42cf5dabfaeb2acfe425e,53f458aadabfaec09f20ed3a,2004,5,53e99784b7602d9701f3f959
1,53f42cf5dabfaeb2acfe425e,53f45a48dabfaee2a1d82ade,2004,5,53e99784b7602d9701f3f959
2,53f458aadabfaec09f20ed3a,53f45a48dabfaee2a1d82ade,2004,5,53e99784b7602d9701f3f959
3,53f8467cdabfae92b411c5df,540fce40dabfae450f4a6981,2009,7,53e99784b7602d9701f3f6bd
4,53f8467cdabfae92b411c5df,54055b62dabfae92b41c9005,2009,7,53e99784b7602d9701f3f6bd


In [4]:
df.drop(labels=['year', 'article_id'], axis=1, inplace=True)
df.drop_duplicates(subset=['author_id_1', 'author_id_2'], inplace=True, ignore_index=True)
print(df.shape)
df.head()


(2315006, 3)


,author_id_1,author_id_2,label
0,53f42cf5dabfaeb2acfe425e,53f458aadabfaec09f20ed3a,5
1,53f42cf5dabfaeb2acfe425e,53f45a48dabfaee2a1d82ade,5
2,53f458aadabfaec09f20ed3a,53f45a48dabfaee2a1d82ade,5
3,53f8467cdabfae92b411c5df,540fce40dabfae450f4a6981,7
4,53f8467cdabfae92b411c5df,54055b62dabfae92b41c9005,7


Создаем таблицу **df1** с обратным взаимодействием "автор-соавтор" и объединяем ее с таблицей **df** , чистим дубликаты. Затем полученную таблицу **df_concat** объеденяем с **data_train** по ключам  ['author_id_1', 'author_id_2'] . Таблица **result** содержит все взаимодействия  "автор-соавтор" и их класс. 

In [5]:
col1=df['author_id_1']
col2=df['author_id_2']

df1=df.copy()
df1['author_id_1'] = col2
df1['author_id_2'] = col1


df_concat = pd.concat([df, df1], ignore_index=True).drop_duplicates(subset=['author_id_1', 'author_id_2'], ignore_index=True)
df_concat.shape

(4628821, 3)

In [ ]:
#for all data
#df_concat1 = pd.concat([data_train, data_test], ignore_index=True).drop_duplicates(subset=['author_id_1', 'author_id_2'], ignore_index=True)
#result = pd.merge(df_concat1, df_concat, on=['author_id_1', 'author_id_2'])


In [6]:
result = pd.merge(data_train, df_concat, on=['author_id_1', 'author_id_2'])
result.head()

,author_id_1,author_id_2,value,label
0,53f42f76dabfaee43ebdfad6,53f452cfdabfaedf435fdb35,2,0
1,53f42f3fdabfaee43ebdd0d1,53f4d3ffdabfaeedcf78223d,1,0
2,53f39223dabfae4b34a5b83f,53f4822ddabfaec09f2a2918,1,18
3,53f43260dabfaec09f153c2b,543407c2dabfaebba5837ff7,1,13
4,53f42d89dabfaedd74d38c8e,53f442e9dabfaeb22f4b29ec,1,16


### 2. Из матрицы **result** выбираем строки одного класса (здесь класс 0)

In [ ]:
class_id = 0
res_class_0 = result[result['label'] == class_id]


,author_id_1,author_id_2,value,label
0,53f42f76dabfaee43ebdfad6,53f452cfdabfaedf435fdb35,2,0
1,53f42f3fdabfaee43ebdd0d1,53f4d3ffdabfaeedcf78223d,1,0
8,53f42b90dabfaec22b9faf07,53f453e6dabfaeb22f4f714b,1,0
22,53f38eb7dabfae4b34a47194,53f437dddabfaeb2ac0609bc,1,0
60,53f4321adabfaec09f1502e6,53f4734edabfaee02adc79a9,1,0


Строим разреженную матрицу взаимодействий авторов

In [ ]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

author_c = CategoricalDtype(res_class_0.author_id_1.unique(), ordered=True)
co_auth_c = CategoricalDtype(res_class_0.author_id_2.unique(), ordered=True)

row = res_class_0.author_id_1.astype(author_c).cat.codes
col = res_class_0.author_id_2.astype(co_auth_c).cat.codes

interactions = csr_matrix((res_class_0["value"], (row, col)), \
                           shape=(author_c.categories.size, co_auth_c.categories.size))



В объектах **author_c**, **co_auth_c** хранятся ID авторов, соответствующие индексам строк и столбцов полученной матрицы

In [ ]:
print("Rows id ", author_c.categories[:10])
print("Columns id ", co_auth_c.categories[:10])

Rows id  Index(['53f42f76dabfaee43ebdfad6', '53f42f3fdabfaee43ebdd0d1',
       '53f42b90dabfaec22b9faf07', '53f38eb7dabfae4b34a47194',
       '53f4321adabfaec09f1502e6', '53f44d72dabfaeee22a12c39',
       '53f42640dabfaeb22f3c4777', '53f43803dabfaedf4358e982',
       '53f47657dabfaee02add3eec', '540fde68dabfae450f4af6d9'],
      dtype='object')
Columns id  Index(['53f452cfdabfaedf435fdb35', '53f4d3ffdabfaeedcf78223d',
       '53f453e6dabfaeb22f4f714b', '53f437dddabfaeb2ac0609bc',
       '53f4734edabfaee02adc79a9', '53f46999dabfaeecd6a193e0',
       '53f47cdbdabfaee43ed4b36d', '562c809845cedb3398c41482',
       '53f47c99dabfaec09f293620', '542a65acdabfae61d49765dd'],
      dtype='object')


#Проверка правильности построения матрицы взаимодействий
Для какого-то автора выведем список его соавторов и проверим соответствие строк и столбцов построенной матрицы взаимодействий ID автора и соавторов

In [ ]:
ex = res_class_0[res_class_0.author_id_1=='53f42f76dabfaee43ebdfad6']
ex

,author_id_1,author_id_2,value,label
0,53f42f76dabfaee43ebdfad6,53f452cfdabfaedf435fdb35,2,0
85196,53f42f76dabfaee43ebdfad6,53f43a74dabfaeecd698573e,1,0
248368,53f42f76dabfaee43ebdfad6,53f434b5dabfaeecd694fc3f,2,0


In [ ]:
id = '53f42f76dabfaee43ebdfad6'
row_id = np.where(np.array(author_c.categories) == id)[0][0]	
col_id = interactions.getrow(row_id).nonzero()[1]
print('co-authors ID in result table ', list(ex.author_id_2))
print('co-authors in interaction matrix ', list(co_auth_c.categories[col_id]))

co-authors ID in result table  ['53f452cfdabfaedf435fdb35', '53f43a74dabfaeecd698573e', '53f434b5dabfaeecd694fc3f']
co-authors in interaction matrix  ['53f452cfdabfaedf435fdb35', '53f43a74dabfaeecd698573e', '53f434b5dabfaeecd694fc3f']


#ВАЖНО!!!

В построенной матрице взаимодействий элементы с одинаковыми значениями ID автора по строке и столбц отсутствуют, так как в исходной таблице не было записей с одинаковыми значениями **author_id_1** и **author_id_1**. Поэтому может случиться такая ситуация, что если есть совместная статья у авторов A, B, C, то их векторное представление будет

       A  B  C
    A| 0  1  1
    B| 1  0  1
    C| 1  1  0

Тогда из-за того, что диаганальные элементы нулевые сходство этих векторов (например, по косинусной близости) будет меньше, чем должно, так как два из трех слагаемых числителя будут обнуляться:

$cos(A,B) = \frac{\sum_i{A_i * B_i}} {\sqrt{A_i^2}\sqrt{B_i^2}} = \frac{0*1 + 1*0 + 1*1}{2} = 1/2$

Если диаганальные элементы задать ненулевыми (так как автор всегда сам с собой взаимодействует при написании любой статьи ), то косинусная близость увеличится:

       A  B  C
    A| 1  1  1
    B| 1  1  1
    C| 1  1  1


$cos(A,B) = \frac{3}{3} = 1$

Исходя из вышесказанного матрица взаимодействий была дополнена взаимодействиями атора с самм собой.

  

In [ ]:
col_id_dict = dict(zip(list(co_auth_c.categories), range(len(co_auth_c.categories))))
for i, row_id in enumerate(author_c.categories):
    if col_id_dict.get(row_id) != None:
        interactions[i, col_id_dict[row_id]] = 1	


/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


##3. Строим матрицу схожести авторов 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

def sparse_cosine_similarity(sparse_matrix):
    out = (sparse_matrix.copy() if type(sparse_matrix) is csr_matrix else
           sparse_matrix.tocsr()).astype(float)
    squared = out.multiply(out)
    sqrt_sum_squared_rows = np.array(np.sqrt(squared.sum(axis=1)))[:, 0]
    row_indices, col_indices = out.nonzero()
    out.data /= sqrt_sum_squared_rows[row_indices]
    return out.dot(out.T)

sim = sparse_cosine_similarity(interactions)



##4. Находим рекомендации соавторов
  **SIM_VAL** - пороговое значения сходства
  
  **REC_NUMBER** - максимальное количество рекомендаций соавторов (минимальное количнство рекомендаций обуславливается параметром SIM_VAL)

In [ ]:
#find the most similar users for each user
SIM_VAL=0.7
REC_NUMBER = 10
sorted_similarity=[]
author_id=[]
for ind, author in enumerate(author_c.categories):
  
  l=list([j for j in sim.getrow(ind).nonzero()[1] if j!=ind and sim[ind,j] >= SIM_VAL])#list of column indeces where elements==True
  if len(l) > REC_NUMBER:
    min_u = REC_NUMBER
  else:
    min_u = len(l)
  
  if len(l)>0:
    tmp1 = [(co_auth_c.categories[j], sim[ind,j]) for j in l[:min_u]]#create temporary list of tuples (id,similarity)
    sorted_similarity.append(sorted(tmp1, key=lambda tup: tup[1], reverse=True))#sort in order to similarity from highest to lowest
    author_id.append(author)



### Матрица рекомендаций:
Индексы строк соответствуют ID автора, элементы в столбцах - tuple ("ID соавтора", "Косинусная близость") 

In [ ]:
sim_df=pd.DataFrame(sorted_similarity,index=author_id)
sim_df.head(10)

,0,1,2,3,4,5,6,7,8,9
53f476d5dabfaee43ed373e6,"(53f7e4f2dabfae9060af5416, 0.981495457622364)",None,None,None,None,None,None,None,None,None
53f45d0ddabfaeb22f519780,"(53f3aa90dabfae4b34af4657, 0.8728715609439696)","(53f44430dabfaee02ad10393, 0.8703882797784892)","(53f42e96dabfaeb22f417ba9, 0.816496580927726)",None,None,None,None,None,None,None
53f44fefdabfaee4dc7f7a6e,"(53f42d3edabfaeb1a7b8904a, 0.7613869876268808)",None,None,None,None,None,None,None,None,None
53f44235dabfaee2a1d26632,"(5432440fdabfaeb4c6a7c001, 0.7970533969860857)",None,None,None,None,None,None,None,None,None
53f42f23dabfaeb22f41ea33,"(53f47384dabfaee4dc88240b, 0.75)","(53f4cfb1dabfaeeee6f80d99, 0.720576692122892)",None,None,None,None,None,None,None,None
53f44ee3dabfaee43ec9f185,"(53f45fe1dabfaec09f226c58, 0.7071067811865475)",None,None,None,None,None,None,None,None,None
53f4309edabfaee2a1ca5cc6,"(54891f47dabfae8a11fb4418, 0.8359173188630291)","(53f4c8c0dabfaee57877e093, 0.8355044182110837)","(54312baadabfae8f29133ee8, 0.8126045083289423)",None,None,None,None,None,None,None
53f46e54dabfaedd74e89591,"(53f4740adabfaee43ed2c11a, 0.8333333333333335)",None,None,None,None,None,None,None,None,None
53f43403dabfaeb22f45a6b2,"(53f46aa6dabfaee02ada5ebd, 0.9015889691237622)","(53f445ccdabfaeb22f4bd342, 0.707549137677225)",None,None,None,None,None,None,None,None
53f4616ddabfaee0d9c1b5c6,"(53f4c6cadabfaee57677b810, 0.7130240959073808)","(562cb6ec45cedb3398ca1d35, 0.704057578618629)",None,None,None,None,None,None,None,None


In [ ]:

sim_df.to_csv('/content/drive/MyDrive/MADE/data/sim_matrix/sim_matrix_class' + str(class_id) + '.csv')

##5. Тестирование рекомендаций

Для тестирования рекомендаций и выбора параметра SIM_VAL мы использовали классификационную метрику **Precision@k**.

Показатели классификации оценивают способность рекомендательных систем принимать решения. Они являются хорошим выбором для таких задач, как определение релевантных или нерелевантных продуктов для пользователя. Для метрик поддержки принятия решений точный рейтинг игнорируется.

  
**Precision@k** - доля топ k рекомендаций, которые релевантны пользователю

$P = \frac{\text{# of  top  k  recommendations  that  are  relevant}}{\text{# of items that are recommended}}$

Ниже приведен код тестирования для разных классов и значений SIM_VAL.

Можно сделать следующие выводы: 

    1. При увеличении значения SIM_VAL значение **Precision@k** растет,
    так как рекомендации становятся более точными, и при этом число всех
    рекоммендаций становиться меньше. Но также уменьшается число тестовых
    авторов, для которых вообще удалось найти подходящих соавторов.

    2. Так как значение **Precision@k** увеличивается незначительно при
    увеличении SIM_VAL, то предпочтительнее взять SIM_VAL меньше, 
    чтобы больше авторов получили рекомендации.

    3. Маленькое значение **Precision@k** обуславливается тем, что 
    взаимодействие ученых в рамках научных исследований (публикаций) является 
    очень "узким", т.е. ученый публикуется единолично, со своими коллегами 
    по вузу или учениками. И разные "научные коллективы" являются достаточно
    обособленными и редко пересекаются друг с другом. Поэтому метрики качества
     рекомендаций на основе колаборативной фильтрации (и на основе графовых 
     структур) будут низкими.  






In [7]:
result_test = pd.merge(data_test, df_concat, on=['author_id_1', 'author_id_2'])


In [8]:
def test_metrics(recom, test, class_num):
  TP=0
  FP=0
  FN=0
  recommendation_count = 0
  user_count = 0
  
  test_authors = np.unique(list(test['author_id_1'])) 
  print("Unique users number =", len(test_authors))
  for i, author in enumerate(test_authors):
      
      co_auths = list(test[test['author_id_1']==author].author_id_2)
      recom_co_auth = recom[recom.index==author]
      if recom_co_auth.shape[0] > 0:
          user_count += 1
          for col in range(recom_co_auth.shape[1]):
              if recom_co_auth.iloc[0, col] != None:
                  recommendation_count += 1
                  if recom_co_auth.iloc[0, col][0] in co_auths:
                      TP += 1
              
     
      if (i % 5000) == 0:
          print(f'Processed {i} authors ...')   
      

  print("class_id", class_id)
  print('Fraction of TEST users with recommendations=', user_count/ len(test_authors))
  print('TP=',TP)
  print('P_k=', TP / recommendation_count )



In [9]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

def sparse_cosine_similarity(sparse_matrix):
    out = (sparse_matrix.copy() if type(sparse_matrix) is csr_matrix else
           sparse_matrix.tocsr()).astype(float)
    squared = out.multiply(out)
    sqrt_sum_squared_rows = np.array(np.sqrt(squared.sum(axis=1)))[:, 0]
    row_indices, col_indices = out.nonzero()
    out.data /= sqrt_sum_squared_rows[row_indices]
    return out.dot(out.T)


#For each class test Precision@k for different values of SIM_VAL
for class_id in range(5):
  print("CLASS_ID ", class_id )

  #select class_id
  res_class_0 = result[result['label'] == class_id]

  #build sparse matrix of interactions
  author_c = CategoricalDtype(res_class_0.author_id_1.unique(), ordered=True)
  co_auth_c = CategoricalDtype(res_class_0.author_id_2.unique(), ordered=True)

  row = res_class_0.author_id_1.astype(author_c).cat.codes
  col = res_class_0.author_id_2.astype(co_auth_c).cat.codes

  interactions = csr_matrix((res_class_0["value"], (row, col)), \
                           shape=(author_c.categories.size, co_auth_c.categories.size))

  #fix interaction matrix
  col_id_dict = dict(zip(list(co_auth_c.categories), range(len(co_auth_c.categories))))
  for i, row_id in enumerate(author_c.categories):
      if col_id_dict.get(row_id) != None:
          interactions[i, col_id_dict[row_id]] = 1	

  #build cosine similrity matrix
  sim = sparse_cosine_similarity(interactions)

  REC_NUMBER = 20

  #find recomendations for different SIM_VAL
  for SIM_VAL in [0.6, 0.7, 0.8]:
    print("SIM_VAL = ", SIM_VAL)
    print(sim.shape[0])
    sorted_similarity=[]
    author_id=[]
  
    for ind, author in enumerate(author_c.categories):
        tmp =list([(author_c.categories[j], sim[ind,j]) for j in sim.getrow(ind).nonzero()[1] if j!=ind and sim[ind,j] >= SIM_VAL])#list of column indeces where elements==True
        
        if len(tmp) > REC_NUMBER:
          min_u = REC_NUMBER
        else:
          min_u = len(tmp)
        
        if len(tmp)>0:
          sort_tmp = sorted(tmp, key=lambda tup: tup[1], reverse=True)
          sorted_similarity.append(sort_tmp[:min_u])#sort in order to similarity from highest to lowest
          author_id.append(author)
          
       

    sim_df=pd.DataFrame(sorted_similarity,index=author_id)
    print("Authors with rec", len(sim_df.index))
    
    # Test recomendations on data_test
    test_class_data = result_test[result_test['label'] == class_id]
    test_metrics(sim_df, test_class_data, class_id)





CLASS_ID  0


/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


SIM_VAL =  0.6
41532
Authors with rec 36423
Unique users number = 27404
Processed 0 authors ...
Processed 5000 authors ...
Processed 10000 authors ...
Processed 15000 authors ...
Processed 20000 authors ...
Processed 25000 authors ...
class_id 0
Fraction of TEST users with recommendations= 0.8433075463435995
TP= 21147
P_k= 0.3042077249514493
SIM_VAL =  0.7
41532
Authors with rec 30967
Unique users number = 27404
Processed 0 authors ...
Processed 5000 authors ...
Processed 10000 authors ...
Processed 15000 authors ...
Processed 20000 authors ...
Processed 25000 authors ...
class_id 0
Fraction of TEST users with recommendations= 0.7137644139541672
TP= 15820
P_k= 0.3149386845039019
SIM_VAL =  0.8
41532
Authors with rec 23006
Unique users number = 27404
Processed 0 authors ...
Processed 5000 authors ...
Processed 10000 authors ...
Processed 15000 authors ...
Processed 20000 authors ...
Processed 25000 authors ...
class_id 0
Fraction of TEST users with recommendations= 0.5244854765727631
TP

#Код для нахождения наиболее частого класса авторов

In [ ]:
author_label = result.drop(labels=['author_id_2','value'], axis=1)

In [ ]:
from collections import Counter
aa=author_label.groupby('author_id_1', sort=False)['label'].agg(lambda srs: Counter(list(srs)).most_common(1)[0][0])#.agg(pd.Series.mode).to_frame()


In [ ]:
aa.to_csv(path+'/author_id_label.csv', header=0)

In [ ]:
a_list=list(aa['label'])
a_list[0]

('53f42f76dabfaee43ebdfad6', 0         0
 138796    0
 404749    0
 404750    0
 Name: label, dtype: int64)

In [ ]:
from collections import Counter
cnt = Counter()
Counter(a_list[0][1].values).most_common(1)[0][0]

0